# Teams

In this section you'll learn how to create a _multi-agent team_ (or simply team) using AutoGen. A team is a group of agents that work together to achieve a common goal.

We'll first show you how to create and run a team. We'll then explain how to observe the team's behavior, which is crucial for debugging and understanding the team's performance, and common operations to control the team's behavior.

```{note}
When should you use a team?
Teams are for complex tasks that require collaboration and diverse expertise.
However, they also demand more scaffolding to steer compared to single agents.
While AutoGen simplifies the process of working with teams, start with
a single agent for simpler tasks, and transition to a multi-agent team when a single agent proves inadequate.
Ensure that you have optimized your single agent with the appropriate tools
and instructions before moving to a team-based approach.
```

## Creating a Team

{py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We will begin by creating a team with two {py:class}`~autogen_agentchat.agents.AssistantAgent` and a {py:class}`~autogen_agentchat.conditions.TextMentionTermination` condition that stops the team when a specific word is detected in the agent's response.

The two-agent team implements the _reflection_ pattern, a multi-agent design pattern where a critic agent evaluates the responses of a primary agent. Learn more about the reflection pattern using the [Core API](../../core-user-guide/design-patterns/reflection.ipynb).

In [14]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team

Let's call the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` method
to start the team with a task.

In [2]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=109), content="Leaves of amber, gold, and rust,  \nDance upon the gentle gust.  \nCrisp air whispers tales of old,  \nAs daylight wanes, the night grows bold.  \n\nPumpkin patch and apple treats,  \nLaughter in the street repeats.  \nSweaters warm and fires aglow,  \nIt's time for nature's vibrant show.  \n\nThe harvest moon ascends the sky,  \nWhile geese in formation start to fly.  \nAutumn speaks in colors bright,  \nA fleeting grace, a pure delight.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=154, completion_tokens=200), content='Your poem beautifully captures the essence of the fall season with vivid imagery and a rhythmic flow. The use of descriptive language like "amber, gold, and rust" effectively pa

The team runs the agents until the termination condition was met.
In this case, the team ran agents following a round-robin order until the the
termination condition was met when the word "APPROVE" was detected in the
agent's response.
When the team stops, it returns a {py:class}`~autogen_agentchat.base.TaskResult` object with all the messages produced by the agents in the team.

## Observing a Team

Similar to the agent's {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` method, you can stream the team's messages while it is running by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the {py:class}`~autogen_agentchat.base.TaskResult` object.

In [4]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=105) content="Leaves descend in golden dance,  \nWhispering secrets as they fall,  \nCrisp air brings a gentle trance,  \nHeralding Autumn's call.  \n\nPumpkins glow with orange light,  \nFields wear a cloak of amber hue,  \nDays retreat to longer night,  \nSkies shift to deeper blue.  \n\nWinds carry scents of earth and pine,  \nSweaters wrap us, warm and tight,  \nNature's canvas, bold design,  \nIn Fall's embrace, we find delight.  " type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=150, completion_tokens=226) content='Your poem beautifully captures the essence of fall with vivid imagery and a soothing rhythm. The imagery of leaves descending, pumpkins glowing, and fields cloaked in amber hues effectively paints a picture of the autumn season. The use of contrasting elements like

As demonstrated in the example above, you can determine the reason why the team stopped by checking the {py:attr}`~autogen_agentchat.base.TaskResult.stop_reason` attribute.

The {py:meth}`~autogen_agentchat.ui.Console` method provides a convenient way to print messages to the console with proper formatting.


In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in crisp air dance,  
Whispering tales as they prance.  
Amber hues paint the ground,  
Nature's symphony all around.  

Sweaters hug with tender grace,  
While pumpkins smile, a warm embrace.  
Chill winds hum through towering trees,  
A vibrant tapestry in the breeze.  

Harvest moons in twilight glow,  
Casting magic on fields below.  
Fall's embrace, a gentle call,  
To savor beauty before snowfalls.  
[Prompt tokens: 28, Completion tokens: 99]
---------- critic ----------
Your poem beautifully captures the essence of the fall season, creating a vivid and cozy atmosphere. The imagery of golden leaves and amber hues paints a picturesque scene that many can easily relate to. I particularly appreciate the personification of pumpkins and the gentle embrace of sweaters, which adds warmth to your verses. 

To enhance the poem further, you might consider adding more sensory det

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=99), content="Golden leaves in crisp air dance,  \nWhispering tales as they prance.  \nAmber hues paint the ground,  \nNature's symphony all around.  \n\nSweaters hug with tender grace,  \nWhile pumpkins smile, a warm embrace.  \nChill winds hum through towering trees,  \nA vibrant tapestry in the breeze.  \n\nHarvest moons in twilight glow,  \nCasting magic on fields below.  \nFall's embrace, a gentle call,  \nTo savor beauty before snowfalls.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=157), content="Your poem beautifully captures the essence of the fall season, creating a vivid and cozy atmosphere. The imagery of golden leaves and amber hues paints a picturesque scene that many can 

## Resetting a Team

You can reset the team by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset` method. This method will clear the team's state, including all agents.
It will call the each agent's {py:meth}`~autogen_agentchat.base.ChatAgent.on_reset` method to clear the agent's state.

In [7]:
await team.reset()  # Reset the team for the next run.

It is usually a good idea to reset the team if the next task is not related to the previous task.
However, if the next task is related to the previous task, you don't need to reset and you can instead
resume the team.

## Stopping a Team

Apart from automatic termination conditions such as {py:class}`~autogen_agentchat.conditions.TextMentionTermination`
that stops the team based on the internal state of the team, you can also stop the team
from outside by using the {py:class}`~autogen_agentchat.conditions.ExternalTermination`.

Calling {py:meth}`~autogen_agentchat.conditions.ExternalTermination.set` 
on {py:class}`~autogen_agentchat.conditions.ExternalTermination` will stop
the team when the current agent's turn is over.
Thus, the team may not stop immediately.
This allows the current agent to finish its turn and broadcast the final message to the team
before the team stops, keeping the team's state consistent.

In [10]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves of amber, gold, and red,  
Gently drifting from trees overhead.  
Whispers of wind through the crisp, cool air,  
Nature's canvas painted with care.  

Harvest moons and evenings that chill,  
Fields of plenty on every hill.  
Sweaters wrapped tight as twilight nears,  
Fall's charming embrace, as warm as it appears.  

Pumpkins aglow with autumn's light,  
Harvest feasts and stars so bright.  
In every leaf and breeze that calls,  
We find the magic of glorious fall.  
[Prompt tokens: 28, Completion tokens: 114]
---------- Summary ----------
Number of messages: 2
Finish reason: External termination requested
Total prompt tokens: 28
Total completion tokens: 114
Duration: 1.71 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=114), content="Leaves of amber, gold, and red,  \nGently drifting from trees overhead.  \nWhispers of wind through the crisp, cool air,  \nNature's canvas painted with care.  \n\nHarvest moons and evenings that chill,  \nFields of plenty on every hill.  \nSweaters wrapped tight as twilight nears,  \nFall's charming embrace, as warm as it appears.  \n\nPumpkins aglow with autumn's light,  \nHarvest feasts and stars so bright.  \nIn every leaf and breeze that calls,  \nWe find the magic of glorious fall.  ", type='TextMessage')], stop_reason='External termination requested')

From the ouput above, you can see the team stopped because the external termination condition was met,
but the speaking agent was able to finish its turn before the team stopped.

## Resuming a Team

Teams are stateful and maintains the conversation history and context
after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` method again
without a new task.
{py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` will continue from the next agent in the round-robin order.

In [11]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- critic ----------
This poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. The descriptions of the changing leaves, cool air, and various autumn traditions make it easy for readers to envision and feel the charm of fall. Here are a few suggestions to enhance its impact:

1. **Structure Variation**: Consider breaking some lines with a hyphen or ellipsis for dramatic effect or emphasis. For instance, “Sweaters wrapped tight as twilight nears— / Fall’s charming embrace, as warm as it appears."

2. **Sensory Details**: While the poem already evokes visual and tactile senses, incorporating other senses such as sound or smell could deepen the immersion. For example, include the scent of wood smoke or the crunch of leaves underfoot.

3. **Metaphorical Language**: Adding metaphors or similes can further enrich the imagery. For example, you might compare the leaves falling to a golden rain or the chill in the air to a gentle whisper.

Ov

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=237), content='This poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. The descriptions of the changing leaves, cool air, and various autumn traditions make it easy for readers to envision and feel the charm of fall. Here are a few suggestions to enhance its impact:\n\n1. **Structure Variation**: Consider breaking some lines with a hyphen or ellipsis for dramatic effect or emphasis. For instance, “Sweaters wrapped tight as twilight nears— / Fall’s charming embrace, as warm as it appears."\n\n2. **Sensory Details**: While the poem already evokes visual and tactile senses, incorporating other senses such as sound or smell could deepen the immersion. For example, include the scent of wood smoke or the crunch of leaves underfoot.\n\n3. **Metaphorical Language**: Adding metaphors or similes can further enrich the imagery. For exampl

You can see the team resumed from where it left off in the output above,
and the first message is from the next agent after the last agent that spoke
before the team stopped.

Let's resume the team again with a new task while keeping the context about the previous task.

In [12]:
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- user ----------
将这首诗用中文唐诗风格写一遍。
---------- primary ----------
朔风轻拂叶飘金，  
枝上斜阳染秋林。  
满山丰收人欢喜，  
月明归途衣渐紧。  

南瓜影映灯火中，  
落叶沙沙伴归程。  
片片秋意随风起，  
秋韵悠悠心自明。  
[Prompt tokens: 700, Completion tokens: 77]
---------- critic ----------
这首改编的唐诗风格诗作成功地保留了原诗的意境与情感，体现出秋季特有的氛围和美感。通过“朔风轻拂叶飘金”、“枝上斜阳染秋林”等意象，生动地描绘出了秋天的景色，与唐诗中的自然意境相呼应。且“月明归途衣渐紧”、“落叶沙沙伴归程”让人感受到秋天的安宁与温暖。

通过这些诗句，读者能够感受到秋天的惬意与宁静，勾起丰收与团圆的画面，是一次成功的翻译改编。

APPROVE
[Prompt tokens: 794, Completion tokens: 161]
---------- Summary ----------
Number of messages: 3
Finish reason: Text 'APPROVE' mentioned
Total prompt tokens: 1494
Total completion tokens: 238
Duration: 3.89 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='将这首诗用中文唐诗风格写一遍。', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=700, completion_tokens=77), content='朔风轻拂叶飘金，  \n枝上斜阳染秋林。  \n满山丰收人欢喜，  \n月明归途衣渐紧。  \n\n南瓜影映灯火中，  \n落叶沙沙伴归程。  \n片片秋意随风起，  \n秋韵悠悠心自明。  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=794, completion_tokens=161), content='这首改编的唐诗风格诗作成功地保留了原诗的意境与情感，体现出秋季特有的氛围和美感。通过“朔风轻拂叶飘金”、“枝上斜阳染秋林”等意象，生动地描绘出了秋天的景色，与唐诗中的自然意境相呼应。且“月明归途衣渐紧”、“落叶沙沙伴归程”让人感受到秋天的安宁与温暖。\n\n通过这些诗句，读者能够感受到秋天的惬意与宁静，勾起丰收与团圆的画面，是一次成功的翻译改编。\n\nAPPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

## Aborting a Team

You can abort a call to {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`
during execution by setting a {py:class}`~autogen_core.CancellationToken` passed to the `cancellation_token` parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a {py:class}`~asyncio.CancelledError` exception.

```{note}
The caller will get a {py:class}`~asyncio.CancelledError` exception when the team is aborted.
```

In [15]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


## Single-Agent Team

Often, you may want to run a single agent in a team configuration.
This is useful for running the {py:class}`~autogen_agentchat.agents.AssistantAgent` in a loop
until a termination condition is met.

This is different from running the {py:class}`~autogen_agentchat.agents.AssistantAgent` using
its {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run` or {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream` method,
which only runs the agent for one step and returns the result.
See {py:class}`~autogen_agentchat.agents.AssistantAgent` for more details about a single step.

Here is an example of running a single agent in a {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` team configuration
with a {py:class}`~autogen_agentchat.conditions.TextMessageTermination` condition.
The task is to increment a number until it reaches 10 using a tool.
The agent will keep calling the tool until the number reaches 10,
and then it will return a final {py:class}`~autogen_agentchat.messages.TextMessage`
which will stop the run.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
    # Disable parallel tool calls for this example.
    parallel_tool_calls=False,  # type: ignore
)


# Create a tool for incrementing a number.
def increment_number(number: int) -> int:
    """Increment a number by 1."""
    return number + 1


# Create a tool agent that uses the increment_number function.
looped_assistant = AssistantAgent(
    "looped_assistant",
    model_client=model_client,
    tools=[increment_number],  # Register the tool.
    system_message="You are a helpful AI assistant, use the tool to increment the number.",
)

# Termination condition that stops the task if the agent responds with a text message.
termination_condition = TextMessageTermination("looped_assistant")

# Create a team with the looped assistant agent and the termination condition.
team = RoundRobinGroupChat(
    [looped_assistant],
    termination_condition=termination_condition,
)

# Run the team with a task and print the messages to the console.
async for message in team.run_stream(task="Increment the number 5 to 10."):  # type: ignore
    print(type(message).__name__, message)

TextMessage source='user' models_usage=None metadata={} content='Increment the number 5 to 10.' type='TextMessage'
ToolCallRequestEvent source='looped_assistant' models_usage=RequestUsage(prompt_tokens=75, completion_tokens=15) metadata={} content=[FunctionCall(id='call_qTDXSouN3MtGDqa8l0DM1ciD', arguments='{"number":5}', name='increment_number')] type='ToolCallRequestEvent'
ToolCallExecutionEvent source='looped_assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='6', name='increment_number', call_id='call_qTDXSouN3MtGDqa8l0DM1ciD', is_error=False)] type='ToolCallExecutionEvent'
ToolCallSummaryMessage source='looped_assistant' models_usage=None metadata={} content='6' type='ToolCallSummaryMessage'
ToolCallRequestEvent source='looped_assistant' models_usage=RequestUsage(prompt_tokens=103, completion_tokens=15) metadata={} content=[FunctionCall(id='call_VGZPlsFVVdyxutR63Yr087pt', arguments='{"number":6}', name='increment_number')] type='ToolCallRequestEvent'

The key is to focus on the termination condition.
In this example, we use a {py:class}`~autogen_agentchat.conditions.TextMessageTermination` condition
that stops the team when the agent stop producing {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage`.
The team will keep running until the agent produces a {py:class}`~autogen_agentchat.messages.TextMessage` with the final result.

You can also use other termination conditions to control the agent.
See [Termination Conditions](./termination.ipynb) for more details.